In [22]:
%cd ../../micropython/ports/unix

/home/v923z/sandbox/micropython/v1.11/micropython/ports/unix


# Introduction

# Usage

The module exposes three functions to python, namely, `fill_rectangle`, `seven_segment`, and `render`. 

## fill_rectangle

The function takes 8 arguments, which are the following:

    1. `memoryview`, a memoryview object, which will contain the pixel information
    2. `colour`, two bytes altogether
    3. `x0`, the x coordinate of the lower left corner of the rectangle
    4. `y0`, the y coordinate of the lower left corner of the rectangle
    5. `width`, the width of the rectangle
    6. `height`, the height of the rectangle
    7. `canvas_width`, the width of the canvas
    8. `canvas_height`, the height of the canvas

## seven_segment

The function takes 6 arguments, namely

    1. `memoryview`, a memoryview object, which will hold the pixel information
    2. `digit`, a single digit of a number 
    3. `size`, the vertical size of the font in pixels
    4. `weight`, the thickness of the font in pixels
    5. `foreground`, the foreground colour, two bytes
    6. `background`, the background colour, two bytes

## render

The function takes the following 5 arguments

    1. `memoryview`, a memoryview object, which will hold the pixel information
    2. `character`, the character description bytearray
    3. `height`, the vertical size of the font in pixels
    4. `foreground`, the foreground colour, two bytes
    5. `background`, the background colour, two bytes

# Notebook conversion

In [65]:
import nbformat as nb
import nbformat.v4.nbbase as nb4
from nbconvert import RSTExporter

def convert_notebook(node, fn):
    (rst, resources) = rstexporter.from_notebook_node(notebook)
    with open(fn, 'w') as fout:
        fout.write(rst)
        
rstexporter = RSTExporter()
rstexporter.template_file = '../../../ufont/docs/templates/rst.tpl'

source = nb.read('../../../ufont/docs/micropython-ufont.ipynb',nb.NO_CONVERT)
i = 0
notebook = nb4.new_notebook()
for j, cell in enumerate(source['cells']):
    if cell['cell_type'] == 'markdown':
        if cell['source'].split('\n')[0].startswith('# '):
            if i >= 1:
                convert_notebook(notebook,'../../../ufont/docs/source/ufont_%02d.rst'%i)
                notebook = nb4.new_notebook()
            i += 1
        notebook.cells.append(cell)
    else:
        notebook.cells.append(cell)
convert_notebook(notebook,'../../../ufont/docs/source/ufont_%02d.rst'%i)

/home/v923z/anaconda3/lib/python3.7/site-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/javascript']) is not able to be represented.
  mimetypes=output.keys())
/home/v923z/anaconda3/lib/python3.7/site-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/javascript']) is not able to be represented.
  mimetypes=output.keys())


# ipython magics

## micropython magic

In [13]:
from IPython.core.magic import Magics, magics_class, line_cell_magic
from IPython.core.magic import cell_magic, register_cell_magic, register_line_magic
import subprocess
import os

Note: if you are trying to run the notebook on windows, you will have to change the destination file accordingly. I chose `/dev/shm/`, so that I won't have to clean up the mess at the end of the session, but any other place should do. 

In [14]:
@register_cell_magic
def micropython(line, cell):
    with open('/dev/shm/micropython.py', 'w') as fout:
        fout.write(cell)
    proc = subprocess.Popen(["./micropython", "/dev/shm/micropython.py"], 
                            stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(proc.stdout.read().decode("utf-8"))
    print(proc.stderr.read().decode("utf-8"))
    return None

Once we have the cell magic, we can run arbitrary micropython commands from the notebook. We should only keep in mind that the header of the cell, the line beginning with `%%micropython`, is not part of the code that is running. 

## C magic

In [15]:
js = """
IPython.CodeCell.options_default.highlight_modes['magic_text/x-csrc'] = {'reg':[/^%%ccode/]};
IPython.notebook.events.one('kernel_ready.Kernel', function(){
    IPython.notebook.get_cells().map(function(cell){
        if (cell.cell_type == 'code'){ cell.auto_highlight(); } }) ;
});
"""
IPython.core.display.display_javascript(js, raw=True)

In [19]:
@magics_class
class MyMagics(Magics):
    @cell_magic
    def ccode(self, line, cell):
        copyright = """/*
 * This file is part of the micropython-ufont project, 
 *
 * https://github.com/v923z/micropython-ufont
 *
 * The MIT License (MIT)
 *
 * Copyright (c) 2020 Zoltán Vörös
*/
    """
        if line:
            with open('../../../ufont/code/'+line, 'w') as cout:
                cout.write(copyright)
                cout.write(cell)
            print('written %d bytes to %s'%(len(copyright) + len(cell), line))
            return None

ip = get_ipython()
ip.register_magics(MyMagics)

# C implementation

## ufont_rectangle.h

In [68]:
%%ccode ufont_rectangle.h

#ifndef _UFONT_RECTANGLE_
#define _UFONT_RECTANGLE_

mp_obj_t ufont_fill_rectangle(size_t , const mp_obj_t *);
mp_obj_t _fill_rectangle(mp_obj_t , mp_obj_t , size_t , size_t , size_t , size_t , size_t , size_t );

#endif

written 403 bytes to ufont_rectangle.h


## ufont_rectangle.c

In [91]:
%%ccode ufont_rectangle.c

#include "py/obj.h"
#include "py/runtime.h"
#include "py/objarray.h"

mp_obj_t _fill_rectangle(mp_obj_t mv, mp_obj_t col, size_t x0, size_t y0, size_t width, size_t height, size_t canvas_width, size_t canvas_height) {
    mp_obj_array_t *memview = MP_OBJ_TO_PTR(mv); // pointer to the memoryview
    if (!MP_OBJ_IS_TYPE(memview, &mp_type_memoryview)) {
        mp_raise_ValueError("first argument must be a memoryview object");
    }
    char *v = memview->items;

    mp_obj_array_t *colour = MP_OBJ_TO_PTR(col); // pointer to the colour definition (2 bytes altogether)
    // TODO: should we allow for bytes, too?
    if ((colour->len != 2) || !MP_OBJ_IS_TYPE(colour, &mp_type_bytearray)) {
        mp_raise_ValueError("second argument must be a two-byte long bytearray");
    }
    char *c = colour->items;

    if((x0 + width > canvas_width) || (y0 + height > canvas_height)) {
        mp_raise_ValueError("rectangle does not fit canvas");
    }

    size_t k;
    for(size_t i=0; i<height; i++) {
        k = 2*(canvas_width*(y0+i)+x0);
        for(size_t j=0; j<width; j++, k+=2) {
            v[k] = c[0];
            v[k+1] = c[1];
        }
    }
    return mp_const_true;
}

mp_obj_t ufont_fill_rectangle(size_t nargs, const mp_obj_t *args) {
    // This is simply a wrapper for _fill_rectangle, so that the function can be exposed in the python interface
    if (nargs != 8) {
        mp_raise_ValueError("function requires 8 arguments");
    }
    size_t x0 = MP_OBJ_SMALL_INT_VALUE(args[2]); // (x0, y0) are the coordinates within the canvas
    size_t y0 = MP_OBJ_SMALL_INT_VALUE(args[3]);
    size_t width = MP_OBJ_SMALL_INT_VALUE(args[4]); // width, and height are the dimensions of the rectangle
    size_t height = MP_OBJ_SMALL_INT_VALUE(args[5]);
    size_t canvas_width = MP_OBJ_SMALL_INT_VALUE(args[6]); // cwidth, and cheight are the dimensions of the canvas
    size_t canvas_height = MP_OBJ_SMALL_INT_VALUE(args[7]);
    return _fill_rectangle(args[0], args[1], x0, y0, width, height, canvas_width, canvas_height);
}

written 2223 bytes to ufont_rectangle.c


## ufont_seven_segment.h

In [27]:
%%ccode ufont_seven_segment.h

#ifndef _UFONT_SEVEN_SEGMENT_
#define _UFONT_SEVEN_SEGMENT_

mp_obj_t ufont_seven_segment(size_t , const mp_obj_t *);

#endif

written 308 bytes to ufont_seven_segment.h


## ufont_seven_segment.c

In [70]:
%%ccode ufont_seven_segment.c

#include "py/obj.h"
#include "py/runtime.h"

#include "ufont_rectangle.h"

mp_obj_t ufont_seven_segment(size_t nargs, const mp_obj_t *args) {
    // Fills up the memoryview in the first argument with the digit given in the second argument
    // The third argument is the height of the font (in pixels), while the fourth is the weight (thickness)
    // The last two arguments are the foreground and background colours, two bytearrays of length at least 2.
    //
    // Example:
    //
    // buffer = bytearray(1000)
    // memview = memoryview(buffer)
    // foreground = bytearray([200, 200])
    // background = bytearray([0, 0])
    // seven_segment(memview, '1', 20, 3, foreground, background)
    //
    // TODO: it might make sense to implement the simple operators +, / and *, as well as the = sign

    if (MP_OBJ_IS_TYPE(args[0], &mp_type_memoryview)) {
        mp_obj_t mv = args[0]; // memoryview
        mp_obj_t fg = args[4]; // foreground colour
        mp_obj_t bg = args[5]; // background colour

        size_t digit, len;
        const byte *str = (const byte*)mp_obj_str_get_data(args[1], &len);
        len = utf8_charlen(str, len);
        if (len == 1) {
            digit = utf8_get_char(str);
        } else {
            mp_raise_ValueError("second argument must be of type character");	
        }

        size_t height = MP_OBJ_SMALL_INT_VALUE(args[2]);
        size_t weight = MP_OBJ_SMALL_INT_VALUE(args[3]); // thickness of the segments
        if (height < 3*weight) {
            mp_raise_ValueError("height must be at least three times as large as weight");
        }
        size_t segment_length = (height-3*weight)/2; // TODO: this number is not necessarily an integer...
        mp_obj_tuple_t *tuple = MP_OBJ_TO_PTR(mp_obj_new_tuple(2, NULL));
        tuple->items[0] = MP_OBJ_NEW_SMALL_INT(height);
        // paint the whole canvas with the background colour
        if (digit == '.') {
            size_t width = weight;
            _fill_rectangle(mv, bg, 0, 0, width, height, width, height); // fill the whole canvas
            _fill_rectangle(mv, fg, 0, height-width, width, width, width, height);
            tuple->items[1] = MP_OBJ_NEW_SMALL_INT(width);
        } else {
            size_t width = 2*weight + segment_length;
            tuple->items[1] = MP_OBJ_NEW_SMALL_INT(width);
            _fill_rectangle(mv, bg, 0, 0, width, height, width, height); // fill the whole canvas
            if (digit == '-') {
                // middle segment
                _fill_rectangle(mv, fg, weight, segment_length+weight, segment_length, weight, width, height);
            } else if (digit == '0') {
                // top segment
                _fill_rectangle(mv, fg, weight, 0, segment_length, weight, width, height);
                // bottom segment
                _fill_rectangle(mv, fg, weight, height-weight, segment_length, weight, width, height);
                // top left
                _fill_rectangle(mv, fg, 0, weight, weight, segment_length, width, height);
                // top right
                _fill_rectangle(mv, fg, weight+segment_length, weight, weight, segment_length, width, height);
                // bottom left
                _fill_rectangle(mv, fg, 0, 2*weight+segment_length, weight, segment_length, width, height);
                // bottom right
                _fill_rectangle(mv, fg, weight+segment_length, 2*weight+segment_length, weight, segment_length, width, height);
            } else if (digit == '1') {
                // top right
                _fill_rectangle(mv, fg, weight+segment_length, weight, weight, segment_length, width, height);
                // bottom right
                _fill_rectangle(mv, fg, weight+segment_length, 2*weight+segment_length, weight, segment_length, width, height);
            } else if (digit == '2') {
                // top segment
                _fill_rectangle(mv, fg, weight, 0, segment_length, weight, width, height);
                // middle segment
                _fill_rectangle(mv, fg, weight, segment_length+weight, segment_length, weight, width, height);
                // bottom segment
                _fill_rectangle(mv, fg, weight, height-weight, segment_length, weight, width, height);
                // top right
                _fill_rectangle(mv, fg, weight+segment_length, weight, weight, segment_length, width, height);
                // bottom left
                _fill_rectangle(mv, fg, 0, 2*weight+segment_length, weight, segment_length, width, height);
            } else if (digit == '3') {
                // top segment
                _fill_rectangle(mv, fg, weight, 0, segment_length, weight, width, height);
                // middle segment
                _fill_rectangle(mv, fg, weight, segment_length+weight, segment_length, weight, width, height);
                // bottom segment
                _fill_rectangle(mv, fg, weight, height-weight, segment_length, weight, width, height);
                // top right
                _fill_rectangle(mv, fg, weight+segment_length, weight, weight, segment_length, width, height);
                // bottom right
                _fill_rectangle(mv, fg, weight+segment_length, 2*weight+segment_length, weight, segment_length, width, height);
            } else if (digit == '4') {
                // middle segment
                _fill_rectangle(mv, fg, weight, segment_length+weight, segment_length, weight, width, height);
                // top left
                _fill_rectangle(mv, fg, 0, weight, weight, segment_length, width, height);
                // top right
                _fill_rectangle(mv, fg, weight+segment_length, weight, weight, segment_length, width, height);
                // bottom right
                _fill_rectangle(mv, fg, weight+segment_length, 2*weight+segment_length, weight, segment_length, width, height);
            } else if (digit == '5') {
                // top segment
                _fill_rectangle(mv, fg, weight, 0, segment_length, weight, width, height);
                // middle segment
                _fill_rectangle(mv, fg, weight, segment_length+weight, segment_length, weight, width, height);
                // bottom segment
                _fill_rectangle(mv, fg, weight, height-weight, segment_length, weight, width, height);
                // top left
                _fill_rectangle(mv, fg, 0, weight, weight, segment_length, width, height);
                // bottom right
                _fill_rectangle(mv, fg, weight+segment_length, 2*weight+segment_length, weight, segment_length, width, height);
            } else if (digit == '6') {
                // top segment
                _fill_rectangle(mv, fg, weight, 0, segment_length, weight, width, height);
                // middle segment
                _fill_rectangle(mv, fg, weight, segment_length+weight, segment_length, weight, width, height);
                // bottom segment
                _fill_rectangle(mv, fg, weight, height-weight, segment_length, weight, width, height);
                // top left
                _fill_rectangle(mv, fg, 0, weight, weight, segment_length, width, height);
                // bottom left
                _fill_rectangle(mv, fg, 0, 2*weight+segment_length, weight, segment_length, width, height);
                // bottom right
                _fill_rectangle(mv, fg, weight+segment_length, 2*weight+segment_length, weight, segment_length, width, height);
            } else if (digit == '7') {
                // top segment
                _fill_rectangle(mv, fg, weight, 0, segment_length, weight, width, height);
                // top right
                _fill_rectangle(mv, fg, weight+segment_length, weight, weight, segment_length, width, height);
                // bottom right
                _fill_rectangle(mv, fg, weight+segment_length, 2*weight+segment_length, weight, segment_length, width, height);
            } else if (digit == '8') {
                // top segment
                _fill_rectangle(mv, fg, weight, 0, segment_length, weight, width, height);
                // middle segment
                _fill_rectangle(mv, fg, weight, segment_length+weight, segment_length, weight, width, height);
                // bottom segment
                _fill_rectangle(mv, fg, weight, height-weight, segment_length, weight, width, height);
                // top left
                _fill_rectangle(mv, fg, 0, weight, weight, segment_length, width, height);
                // top right
                _fill_rectangle(mv, fg, weight+segment_length, weight, weight, segment_length, width, height);
                // bottom left
                _fill_rectangle(mv, fg, 0, 2*weight+segment_length, weight, segment_length, width, height);
                // bottom right
                _fill_rectangle(mv, fg, weight+segment_length, 2*weight+segment_length, weight, segment_length, width, height);
            } else if (digit == '9') {
                // top segment
                _fill_rectangle(mv, fg, weight, 0, segment_length, weight, width, height);
                // middle segment
                _fill_rectangle(mv, fg, weight, segment_length+weight, segment_length, weight, width, height);
                // bottom segment
                _fill_rectangle(mv, fg, weight, height-weight, segment_length, weight, width, height);
                // top left
                _fill_rectangle(mv, fg, 0, weight, weight, segment_length, width, height);
                // top right
                _fill_rectangle(mv, fg, weight+segment_length, weight, weight, segment_length, width, height);
                // bottom right
                _fill_rectangle(mv, fg, weight+segment_length, 2*weight+segment_length, weight, segment_length, width, height);
            }
        }
        return MP_OBJ_FROM_PTR(tuple);
    }
    mp_raise_TypeError("wrong argument type");
}

written 10080 bytes to ufont_seven_segment.c


## ufont_renderer.h

In [31]:
%%ccode ufont_renderer.h

#ifndef _UFONT_RENDERER_
#define _UFONT_RENDERER_

mp_obj_t ufont_render(size_t , const mp_obj_t *);

#endif

written 291 bytes to ufont_renderer.h


## ufont_renderer.c

In [76]:
%%ccode ufont_renderer.c

#include "py/obj.h"
#include "py/runtime.h"
#include "py/objarray.h"

mp_obj_t ufont_render(size_t nargs, const mp_obj_t *args) {
    // Unpacks a pattern into a binary array. The function is meant as a fast implementation of 
    // https://github.com/peterhinch/micropython-font-to-py/blob/master/writer/writer.py
    //
    // Arguments are
    //
    // 		output buffer (memoryview)
    // 		pattern (bytearray/memoryview)
    // 		height (font height, integer, pixels)
    // 		foreground colour (2-byte bytearray)
    // 		background colour (2-byte bytearray)
    //  
    // If a bit is set in 'pattern', then the corresponding two bytes in the output buffer 
    // will be set to the two bytes given by 'foreground', while in the opposite 
    // case, the two bytes will be taken from 'background'. 
    //
    // Example
    //
    // 	buffer = bytearray(1000)
    // 	memview = memoryview(buffer)
    //  pat = bytearray([1, 2, 3, 4, 5]) # this is 40 bits altogether
    //  pattern = memoryview(pat)
    //  height = 10 # this particular font is 4 pixels wide
    // 	foreground = bytearray([200, 200])
    // 	background = bytearray([0, 0])
    // 	render(memview, pattern, height, foreground, background)

    // TODO: in principle, args[1], args[3], and args[4] could be bytes. Should we allow for that?
    if (!mp_obj_is_type(args[0], &mp_type_memoryview) || !mp_obj_is_type(args[1], &mp_type_bytearray) 
        || !mp_obj_is_type(args[3], &mp_type_bytearray) || !mp_obj_is_type(args[4], &mp_type_bytearray)) {
        mp_raise_TypeError("wrong argument type");		
    }

    mp_obj_array_t *mv = MP_OBJ_TO_PTR(args[0]);
    mp_obj_array_t *pat = MP_OBJ_TO_PTR(args[1]);
    size_t height = MP_OBJ_SMALL_INT_VALUE(args[2]);
    mp_obj_array_t *fg = MP_OBJ_TO_PTR(args[3]);
    mp_obj_array_t *bg = MP_OBJ_TO_PTR(args[4]);
    if ((fg->len != 2) || (bg->len != 2)) {
        mp_raise_ValueError("foreground/background pattern should be two bytes long");
    }
    if (mv->len < 2*8*pat->len) {
        // Each byte in the pattern holds the description of 8 pixels, and each pixel requires two bytes
        mp_raise_ValueError("buffer length should be larger than pattern length times 16");
    }
    size_t i, j, k=0;
    char pattern_byte;
    char *p = pat->items;
    char *m = mv->items;
    char *fore = fg->items;
    char *back = bg->items;

    for (i=0; k < (pat->len*16/height)*height; i++) {
        char mask = 0b10000000;
        pattern_byte = p[i];
        for (j=0; j < 8; j++) {
            if (mask & pattern_byte) {
                m[k] = fore[0];
                m[k+1] = fore[1];
            } else {
                m[k] = back[0];
                m[k+1] = back[1];
            }
            mask >>= 1;
            k += 2;
        }
    }
    return mp_const_true;
}

written 2990 bytes to ufont_renderer.c


## ufont.c

In [86]:
%%ccode ufont.c

#include "py/obj.h"

#include "ufont_rectangle.h"
#include "ufont_seven_segment.h"
#include "ufont_renderer.h"

#define UFONT_VERSION 0.99

typedef struct _mp_obj_float_t {
    mp_obj_base_t base;
    mp_float_t value;
} mp_obj_float_t;

mp_obj_float_t ufont_version = {{&mp_type_float}, UFONT_VERSION};

MP_DEFINE_CONST_FUN_OBJ_VAR_BETWEEN(ufont_seven_segment_obj, 6, 6, ufont_seven_segment);
MP_DEFINE_CONST_FUN_OBJ_VAR_BETWEEN(ufont_fill_rectangle_obj, 8, 8, ufont_fill_rectangle);
MP_DEFINE_CONST_FUN_OBJ_VAR_BETWEEN(ufont_render_obj, 5, 5, ufont_render);

STATIC const mp_map_elem_t ufont_globals_table[] = {
    { MP_OBJ_NEW_QSTR(MP_QSTR___name__), MP_OBJ_NEW_QSTR(MP_QSTR_ufont) },
    { MP_ROM_QSTR(MP_QSTR___version__), MP_ROM_PTR(&ufont_version) },
    { MP_OBJ_NEW_QSTR(MP_QSTR_fill_rectangle), (mp_obj_t)&ufont_fill_rectangle_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_seven_segment), (mp_obj_t)&ufont_seven_segment_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_render), (mp_obj_t)&ufont_render_obj }
};

STATIC MP_DEFINE_CONST_DICT (
    mp_module_ufont_globals,
    ufont_globals_table
);

const mp_obj_module_t ufont_user_cmodule = {
    .base = { &mp_type_module },
    .globals = (mp_obj_dict_t*)&mp_module_ufont_globals,
};

MP_REGISTER_MODULE(MP_QSTR_ufont, ufont_user_cmodule, MODULE_UFONT_ENABLED);

written 1487 bytes to ufont.c


# Compilation components

## makefile fragment

In [46]:
%%writefile ../../../ufont/code/micropython.mk

USERMODULES_DIR := $(USERMOD_DIR)

# Add all C files to SRC_USERMOD.
SRC_USERMOD += $(USERMODULES_DIR)/ufont_rectangle.c
SRC_USERMOD += $(USERMODULES_DIR)/ufont_seven_segment.c
SRC_USERMOD += $(USERMODULES_DIR)/ufont_renderer.c
SRC_USERMOD += $(USERMODULES_DIR)/ufont.c
CFLAGS_USERMOD += -I$(USERMODULES_DIR)

Overwriting ../../../ufont/code/micropython.mk


## unix port make

In [90]:
%cd ../unix/

/home/v923z/sandbox/micropython/v1.11/micropython/ports/unix


In [ ]:
!make clean
!make USER_C_MODULES=../../../ufont CFLAGS_EXTRA=-DMODULE_UFONT_ENABLED=1 all

In [88]:
%%micropython -unix 1

import ufont
print(ufont.__version__)

0.99




## pyboard make

In [89]:
%cd ../stm32/

/home/v923z/sandbox/micropython/v1.11/micropython/ports/stm32


In [ ]:
!make clean
!make USER_C_MODULES=../../../ufont CFLAGS_EXTRA=-DMODULE_UFONT_ENABLED=1 all

# Fonts

In [ ]:
%%micropython -pyboard 1

_font =\
b'\x0b\x00\x00\x00\x3c\x00\x7e\x00\xc7\x00\xc3\x00\x03\x00\x03\x00'\
b'\x06\x00\x0c\x00\x08\x00\x18\x00\x18\x00\x00\x00\x00\x00\x18\x00'\
b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x07\x00\x00\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\x00\x00'\
b'\xc0\xc0\x00\x00\x00\x00\x07\x00\x00\x00\xd8\xd8\xd8\xd8\x90\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00'\
b'\x00\x00\x0c\xc0\x08\x80\x08\x80\x7f\xe0\x7f\xe0\x19\x80\x11\x00'\
b'\x11\x00\xff\xc0\xff\xc0\x33\x00\x33\x00\x22\x00\x22\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x0b\x00\x08\x00\x3e\x00\x7f\x80\xe9\xc0'\
b'\xc8\xc0\xc8\xc0\xc8\x00\xe8\x00\x7c\x00\x1f\x80\x09\xc0\x08\xc0'\
b'\xc8\xc0\xe9\xc0\x7f\x80\x3e\x00\x08\x00\x00\x00\x00\x00\x00\x00'\
b'\x12\x00\x00\x00\x00\x00\x00\x00\x38\x10\x00\x7c\x10\x00\xc6\x20'\
b'\x00\xc6\x20\x00\xc6\x40\x00\x7c\xc0\x00\x38\x80\x00\x01\x1e\x00'\
b'\x01\x3f\x00\x02\x73\x80\x02\x61\x80\x04\x73\x80\x04\x3f\x00\x08'\
b'\x1e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0d\x00'\
b'\x00\x00\x00\x00\x0e\x00\x1f\x00\x31\x80\x31\x80\x31\x80\x1f\x00'\
b'\x1c\x00\x76\x60\xe3\x60\xc1\xc0\xc0\xc0\xe1\xc0\x7f\x60\x3e\x30'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\xc0\xc0\xc0\xc0'\
b'\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x07\x00'\
b'\x00\x10\x10\x20\x20\x60\x40\xc0\xc0\xc0\xc0\xc0\xc0\xc0\x40\x60'\
b'\x20\x30\x10\x18\x07\x00\x00\x40\x40\x20\x20\x30\x10\x18\x18\x18'\
b'\x18\x18\x18\x18\x10\x30\x20\x60\x40\xc0\x08\x00\x00\x20\x20\xf8'\
b'\x20\x50\x50\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x0c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x18\x00\x18\x00\x18\x00\xff\x00\xff\x00\x18\x00\x18\x00\x18\x00'\
b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\xc0\x40\x40\x80\x00'\
b'\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf8\xf8\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\xc0\xc0\x00\x00\x00\x00\x06\x00\x00\x04'\
b'\x0c\x08\x08\x18\x10\x10\x30\x20\x20\x60\x40\x40\xc0\x80\x00\x00'\
b'\x00\x00\x0b\x00\x00\x00\x00\x00\x3e\x00\x7f\x00\x63\x00\xe3\x80'\
b'\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xe3\x80\x63\x00'\
b'\x7f\x00\x3e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00'\
b'\x00\x00\x10\x00\x30\x00\xf0\x00\xf0\x00\x30\x00\x30\x00\x30\x00'\
b'\x30\x00\x30\x00\x30\x00\x30\x00\x30\x00\x30\x00\x30\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x3e\x00\x7f\x00'\
b'\xe3\x80\xc1\x80\x01\x80\x01\x80\x03\x00\x0e\x00\x1c\x00\x30\x00'\
b'\x60\x00\xc0\x00\xff\x80\xff\x80\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x0b\x00\x00\x00\x00\x00\x3e\x00\x7f\x00\xe3\x80\xc1\x80\x01\x80'\
b'\x0f\x00\x0f\x00\x03\x80\x01\x80\x01\x80\xc1\x80\xe3\x80\x7f\x00'\
b'\x3e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00\x00\x00'\
b'\x06\x00\x06\x00\x0e\x00\x1e\x00\x16\x00\x26\x00\x46\x00\x46\x00'\
b'\x86\x00\xff\x00\xff\x00\x06\x00\x06\x00\x06\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x7f\x00\x7f\x00\x60\x00'\
b'\x60\x00\xde\x00\xff\x00\xe3\x80\x01\x80\x01\x80\x01\x80\x01\x80'\
b'\xc3\x00\x7f\x00\x3e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x00'\
b'\x00\x00\x00\x00\x1e\x00\x3f\x00\x63\x00\x61\x80\xc0\x00\xde\x00'\
b'\xff\x00\xe3\x80\xc1\x80\xc1\x80\xc1\x80\x63\x80\x7f\x00\x3e\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00\x00\x00\xff\x80'\
b'\xff\x80\x01\x00\x03\x00\x02\x00\x06\x00\x04\x00\x0c\x00\x08\x00'\
b'\x18\x00\x18\x00\x10\x00\x30\x00\x30\x00\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\x0b\x00\x00\x00\x00\x00\x1c\x00\x3e\x00\x63\x00\x63\x00'\
b'\x63\x00\x3e\x00\x3e\x00\x63\x00\xc1\x80\xc1\x80\xc1\x80\x63\x00'\
b'\x7f\x00\x1c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00'\
b'\x00\x00\x3e\x00\x7f\x00\xe3\x00\xc1\x80\xc1\x80\xc1\x80\xe3\x80'\
b'\x7f\x80\x3d\x80\x01\x80\x03\x00\xe3\x00\x7e\x00\x3c\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\xc0\xc0\x00'\
b'\x00\x00\x00\x00\x00\x00\xc0\xc0\x00\x00\x00\x00\x05\x00\x00\x00'\
b'\x00\x00\x00\x00\xc0\xc0\x00\x00\x00\x00\x00\x00\xc0\xc0\x40\x40'\
b'\x80\x00\x0c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x00\x40\x01\xc0\x07\x00\x3c\x00\xe0\x00\xe0\x00\x78\x00\x0f\x00'\
b'\x03\xc0\x00\x40\x00\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xc0'\
b'\xff\xc0\x00\x00\x00\x00\xff\xc0\xff\xc0\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\xe0\x00\x78\x00\x0e\x00\x03\xc0\x01\xc0'\
b'\x07\x00\x3c\x00\xf0\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x0b\x00\x00\x00\x3c\x00\x7e\x00\xc7\x00\xc3\x00\x03\x00\x03\x00'\
b'\x06\x00\x0c\x00\x08\x00\x18\x00\x18\x00\x00\x00\x00\x00\x18\x00'\
b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x03'\
b'\xf0\x00\x0f\xfc\x00\x1e\x0f\x00\x38\x03\x80\x71\xe1\x80\x63\xe9'\
b'\xc0\x67\x18\xc0\xce\x18\xc0\xcc\x18\xc0\xcc\x10\xc0\xcc\x31\x80'\
b'\xce\x73\x80\x67\xff\x00\x63\x9e\x00\x30\x00\x00\x3c\x00\x00\x0f'\
b'\xf8\x00\x03\xf0\x00\x00\x00\x00\x0d\x00\x00\x00\x07\x00\x07\x00'\
b'\x07\x80\x0d\x80\x0d\x80\x08\xc0\x18\xc0\x18\xc0\x10\x60\x3f\xe0'\
b'\x3f\xe0\x30\x30\x60\x30\x60\x38\xc0\x18\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\x0d\x00\x00\x00\xff\x00\xff\x80\xc1\xc0\xc0\xc0\xc0\xc0'\
b'\xc1\xc0\xff\x00\xff\x80\xc0\xc0\xc0\x60\xc0\x60\xc0\x60\xc0\xe0'\
b'\xff\xc0\xff\x80\x00\x00\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00'\
b'\x0f\x80\x3f\xe0\x70\x60\x60\x30\xe0\x00\xc0\x00\xc0\x00\xc0\x00'\
b'\xc0\x00\xc0\x00\xe0\x30\x60\x70\x70\x60\x3f\xe0\x0f\x80\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00\xff\x00\xff\x80\xc1\xc0'\
b'\xc0\xc0\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xc0\x60'\
b'\xc0\xc0\xc1\xc0\xff\x80\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x0d\x00\x00\x00\xff\xc0\xff\xc0\xc0\x00\xc0\x00\xc0\x00\xc0\x00'\
b'\xff\x80\xff\x80\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xff\xc0'\
b'\xff\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00\xff\x80'\
b'\xff\x80\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xff\x00\xff\x00\xc0\x00'\
b'\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x0f\x00\x00\x00\x0f\xc0\x3f\xf0\x38\x30\x60\x18'\
b'\x60\x00\xc0\x00\xc0\x00\xc1\xf8\xc1\xf8\xc0\x18\xe0\x18\x60\x38'\
b'\x78\x78\x3f\xd8\x0f\x88\x00\x00\x00\x00\x00\x00\x00\x00\x0e\x00'\
b'\x00\x00\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xff\xe0'\
b'\xff\xe0\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xc0\x60'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\xc0\xc0\xc0\xc0\xc0'\
b'\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\x00\x00\x00\x00\x0b\x00'\
b'\x00\x00\x03\x00\x03\x00\x03\x00\x03\x00\x03\x00\x03\x00\x03\x00'\
b'\x03\x00\x03\x00\x03\x00\xc3\x00\xc3\x00\xe7\x00\x7e\x00\x3c\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x0d\x00\x00\x00\xc0\x60\xc0\xc0'\
b'\xc1\x80\xc3\x00\xc6\x00\xcc\x00\xdc\x00\xf6\x00\xe6\x00\xc3\x00'\
b'\xc1\x80\xc1\x80\xc0\xc0\xc0\x60\xc0\x60\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\x0b\x00\x00\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00'\
b'\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00'\
b'\xff\x80\xff\x80\x00\x00\x00\x00\x00\x00\x00\x00\x11\x00\x00\x00'\
b'\x00\xe0\x1c\x00\xe0\x1c\x00\xf0\x3c\x00\xf0\x3c\x00\xd0\x2c\x00'\
b'\xd8\x6c\x00\xd8\x6c\x00\xc8\x4c\x00\xcc\xcc\x00\xcc\xcc\x00\xc4'\
b'\x8c\x00\xc6\x8c\x00\xc7\x8c\x00\xc3\x0c\x00\xc3\x0c\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\xe0\x60'\
b'\xe0\x60\xf0\x60\xf0\x60\xd8\x60\xd8\x60\xcc\x60\xc4\x60\xc6\x60'\
b'\xc2\x60\xc3\x60\xc1\xe0\xc1\xe0\xc0\xe0\xc0\xe0\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x10\x00\x00\x00\x0f\xc0\x1f\xe0\x38\x70\x60\x18'\
b'\x60\x1c\xc0\x0c\xc0\x0c\xc0\x0c\xc0\x0c\xc0\x0c\x60\x1c\x60\x18'\
b'\x38\x70\x1f\xe0\x0f\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x0d\x00'\
b'\x00\x00\xff\x00\xff\x80\xc1\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc1\xc0'\
b'\xff\x80\xff\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x0f\xc0\x1f\xe0'\
b'\x38\x70\x60\x18\x60\x1c\xc0\x0c\xc0\x0c\xc0\x0c\xc0\x0c\xc0\x0c'\
b'\x60\x18\x60\xd8\x38\x70\x1f\xf8\x0f\x98\x00\x08\x00\x00\x00\x00'\
b'\x00\x00\x0e\x00\x00\x00\xff\x80\xff\xc0\xc0\xe0\xc0\x60\xc0\x60'\
b'\xc0\x60\xc0\xc0\xff\x80\xff\xc0\xc0\xe0\xc0\x60\xc0\x60\xc0\x60'\
b'\xc0\x60\xc0\x70\x00\x00\x00\x00\x00\x00\x00\x00\x0d\x00\x00\x00'\
b'\x1f\x80\x7f\xe0\xe0\x70\xc0\x30\xc0\x00\xe0\x00\x78\x00\x3f\x80'\
b'\x03\xe0\x00\x70\xc0\x30\xc0\x30\x70\x60\x7f\xe0\x1f\x80\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x0d\x00\x00\x00\xff\xc0\xff\xc0\x0c\x00'\
b'\x0c\x00\x0c\x00\x0c\x00\x0c\x00\x0c\x00\x0c\x00\x0c\x00\x0c\x00'\
b'\x0c\x00\x0c\x00\x0c\x00\x0c\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x0e\x00\x00\x00\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xc0\x60'\
b'\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xc0\x60\xc0\x60\x60\xc0\x7f\xc0'\
b'\x1f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0d\x00\x00\x00\xc0\x30'\
b'\x60\x30\x60\x30\x20\x20\x30\x60\x30\x60\x10\x40\x18\xc0\x18\xc0'\
b'\x08\x80\x0d\x80\x0d\x80\x07\x00\x07\x00\x07\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x13\x00\x00\x00\x00\xc0\xc0\xc0\x60\xe0\xc0\x60'\
b'\xe0\xc0\x61\xe0\xc0\x61\xb1\x80\x31\xb1\x80\x31\xb1\x80\x33\x11'\
b'\x80\x33\x19\x00\x13\x1b\x00\x1f\x1b\x00\x1e\x0b\x00\x1e\x0e\x00'\
b'\x0e\x0e\x00\x0c\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\x0d\x00\x00\x00\x60\x30\x30\x70\x30\x60\x18\xc0\x0c\xc0'\
b'\x0d\x80\x07\x00\x07\x00\x07\x00\x0d\x80\x18\xc0\x18\xe0\x30\x60'\
b'\x70\x30\x60\x38\x00\x00\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00'\
b'\x60\x18\x70\x38\x30\x30\x18\x60\x18\x60\x0c\xc0\x0f\xc0\x07\x80'\
b'\x03\x00\x03\x00\x03\x00\x03\x00\x03\x00\x03\x00\x03\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00\xff\xe0\xff\xe0\x00\xc0'\
b'\x01\x80\x03\x80\x03\x00\x06\x00\x0c\x00\x1c\x00\x38\x00\x30\x00'\
b'\x60\x00\xc0\x00\xff\xe0\xff\xe0\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x06\x00\x00\xe0\xe0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0'\
b'\xc0\xc0\xc0\xc0\xe0\xe0\x06\x00\x00\x80\xc0\x40\x40\x60\x20\x20'\
b'\x30\x10\x10\x18\x08\x08\x0c\x04\x00\x00\x00\x00\x06\x00\x00\xe0'\
b'\xe0\x60\x60\x60\x60\x60\x60\x60\x60\x60\x60\x60\x60\x60\x60\x60'\
b'\xe0\xe0\x09\x00\x00\x00\x00\x00\x18\x00\x38\x00\x28\x00\x2c\x00'\
b'\x64\x00\x46\x00\xc2\x00\x82\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xf0'\
b'\x00\x00\x00\x00\x00\x00\x05\x00\x00\xc0\x60\x30\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x3e\x00\xff\x80\xc1\x80\x01\x80'\
b'\x01\x80\x3f\x80\xf1\x80\xc1\x80\xc3\x80\xff\xc0\x78\xc0\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00\xc0\x00\xc0\x00\xc0\x00'\
b'\xc0\x00\xdf\x00\xff\x80\xe1\x80\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0'\
b'\xc0\xc0\xe1\x80\xff\x80\xde\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x0a\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x1e\x00\x7f\x00'\
b'\x61\x80\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xc1\x80\x63\x80\x7f\x00'\
b'\x3e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00\x01\x80'\
b'\x01\x80\x01\x80\x01\x80\x3d\x80\x7f\x80\x63\x80\xc1\x80\xc1\x80'\
b'\xc1\x80\xc1\x80\xc1\x80\x63\x80\x7f\x80\x3d\x80\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x3e\x00\x7f\x00\x63\x00\xc1\x80\xff\x80\xff\x80\xc0\x00\xc0\x00'\
b'\x63\x80\x7f\x00\x3e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x06\x00'\
b'\x00\x30\x70\x60\x60\xf0\xf0\x60\x60\x60\x60\x60\x60\x60\x60\x60'\
b'\x00\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x3d\x80\x7f\x80\x63\x80\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xc1\x80'\
b'\x63\x80\x7f\x80\x3d\x80\x01\x80\xc3\x80\x7f\x00\x3e\x00\x0b\x00'\
b'\x00\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00\xdf\x00\xdf\x80\xe3\x80'\
b'\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xc1\x80'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\xc0\xc0\x00\x00\xc0'\
b'\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\x00\x00\x00\x00\x05\x00'\
b'\x00\x30\x30\x00\x00\x30\x30\x30\x30\x30\x30\x30\x30\x30\x30\x30'\
b'\x30\x30\xf0\xe0\x0a\x00\x00\x00\xc0\x00\xc0\x00\xc0\x00\xc0\x00'\
b'\xc3\x00\xc6\x00\xcc\x00\xd8\x00\xf8\x00\xec\x00\xce\x00\xc6\x00'\
b'\xc3\x00\xc3\x00\xc1\x80\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00'\
b'\x00\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0'\
b'\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\xde\x78\xfe\xfc\xe3\x8c\xc3\x0c\xc3\x0c\xc3\x0c\xc3\x0c\xc3\x0c'\
b'\xc3\x0c\xc3\x0c\xc3\x0c\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcf\x00\xdf\x80\xe3\x80'\
b'\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xc1\x80'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x3e\x00\x7f\x00\x63\x00\xc1\x80\xc1\x80\xc1\x80'\
b'\xc1\x80\xc1\x80\x63\x00\x7f\x00\x3e\x00\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xde\x00'\
b'\xff\x80\xe1\x80\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xe1\x80'\
b'\xff\x80\xde\x00\xc0\x00\xc0\x00\xc0\x00\x00\x00\x0b\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x3d\x80\x7f\x80\x63\x80\xc1\x80'\
b'\xc1\x80\xc1\x80\xc1\x80\xc1\x80\x63\x80\x7f\x80\x3d\x80\x01\x80'\
b'\x01\x80\x01\x80\x00\x00\x07\x00\x00\x00\x00\x00\x00\xd8\xf8\xe0'\
b'\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\x00\x00\x00\x00\x0a\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x3c\x00\x7f\x00\xc3\x00\xc0\x00'\
b'\xf0\x00\x7e\x00\x0f\x00\x03\x00\xc3\x00\xfe\x00\x7c\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x60\x60\xf0\xf0\x60'\
b'\x60\x60\x60\x60\x60\x60\x70\x70\x00\x00\x00\x00\x0b\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\xc1\x80\xc1\x80\xc1\x80\xc1\x80'\
b'\xc1\x80\xc1\x80\xc1\x80\xc1\x80\xe3\x80\xfd\x80\x79\x80\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x0a\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\xc0\xc0\x61\x80\x61\x80\x61\x00\x23\x00\x33\x00\x32\x00'\
b'\x16\x00\x1e\x00\x1c\x00\x0c\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x0e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc3\x0c\xc3\x8c'\
b'\x63\x8c\x67\x88\x66\x98\x24\xd8\x34\xd0\x3c\xd0\x3c\x70\x18\x70'\
b'\x18\x60\x00\x00\x00\x00\x00\x00\x00\x00\x0a\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x61\x80\x63\x00\x33\x00\x1e\x00\x1c\x00'\
b'\x0c\x00\x1c\x00\x16\x00\x33\x00\x63\x00\x41\x80\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x0a\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\xc0\x80\x41\x80\x61\x80\x61\x00\x23\x00\x33\x00\x32\x00\x16\x00'\
b'\x1c\x00\x1c\x00\x0c\x00\x08\x00\x18\x00\x78\x00\x70\x00\x0a\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\x00\xff\x00\x06\x00'\
b'\x06\x00\x0c\x00\x18\x00\x30\x00\x60\x00\xc0\x00\xff\x00\xff\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x18\x38\x30\x30\x30'\
b'\x30\x30\x30\x70\xc0\x70\x30\x30\x30\x30\x30\x30\x38\x18\x05\x00'\
b'\x00\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0\xc0'\
b'\xc0\xc0\xc0\xc0\x07\x00\x00\xc0\xe0\x60\x60\x60\x60\x60\x60\x70'\
b'\x18\x70\x60\x60\x60\x60\x60\x60\xe0\xc0\x0a\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x60\x00\xf1\x00\x9f\x00'\
b'\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'\
b'\x00\x00\x00\x00'

_index =\
b'\x00\x00\x2a\x00\x2a\x00\x40\x00\x40\x00\x56\x00\x56\x00\x6c\x00'\
b'\x6c\x00\x96\x00\x96\x00\xc0\x00\xc0\x00\xfe\x00\xfe\x00\x28\x01'\
b'\x28\x01\x3e\x01\x3e\x01\x54\x01\x54\x01\x6a\x01\x6a\x01\x80\x01'\
b'\x80\x01\xaa\x01\xaa\x01\xc0\x01\xc0\x01\xd6\x01\xd6\x01\xec\x01'\
b'\xec\x01\x02\x02\x02\x02\x2c\x02\x2c\x02\x56\x02\x56\x02\x80\x02'\
b'\x80\x02\xaa\x02\xaa\x02\xd4\x02\xd4\x02\xfe\x02\xfe\x02\x28\x03'\
b'\x28\x03\x52\x03\x52\x03\x7c\x03\x7c\x03\xa6\x03\xa6\x03\xbc\x03'\
b'\xbc\x03\xd2\x03\xd2\x03\xfc\x03\xfc\x03\x26\x04\x26\x04\x50\x04'\
b'\x50\x04\x7a\x04\x7a\x04\xb8\x04\xb8\x04\xe2\x04\xe2\x04\x0c\x05'\
b'\x0c\x05\x36\x05\x36\x05\x60\x05\x60\x05\x8a\x05\x8a\x05\xb4\x05'\
b'\xb4\x05\xde\x05\xde\x05\x08\x06\x08\x06\x1e\x06\x1e\x06\x48\x06'\
b'\x48\x06\x72\x06\x72\x06\x9c\x06\x9c\x06\xda\x06\xda\x06\x04\x07'\
b'\x04\x07\x2e\x07\x2e\x07\x58\x07\x58\x07\x82\x07\x82\x07\xac\x07'\
b'\xac\x07\xd6\x07\xd6\x07\x00\x08\x00\x08\x2a\x08\x2a\x08\x54\x08'\
b'\x54\x08\x92\x08\x92\x08\xbc\x08\xbc\x08\xe6\x08\xe6\x08\x10\x09'\
b'\x10\x09\x26\x09\x26\x09\x3c\x09\x3c\x09\x52\x09\x52\x09\x7c\x09'\
b'\x7c\x09\xa6\x09\xa6\x09\xbc\x09\xbc\x09\xe6\x09\xe6\x09\x10\x0a'\
b'\x10\x0a\x3a\x0a\x3a\x0a\x64\x0a\x64\x0a\x8e\x0a\x8e\x0a\xa4\x0a'\
b'\xa4\x0a\xce\x0a\xce\x0a\xf8\x0a\xf8\x0a\x0e\x0b\x0e\x0b\x24\x0b'\
b'\x24\x0b\x4e\x0b\x4e\x0b\x64\x0b\x64\x0b\x8e\x0b\x8e\x0b\xb8\x0b'\
b'\xb8\x0b\xe2\x0b\xe2\x0b\x0c\x0c\x0c\x0c\x36\x0c\x36\x0c\x4c\x0c'\
b'\x4c\x0c\x76\x0c\x76\x0c\x8c\x0c\x8c\x0c\xb6\x0c\xb6\x0c\xe0\x0c'\
b'\xe0\x0c\x0a\x0d\x0a\x0d\x34\x0d\x34\x0d\x5e\x0d\x5e\x0d\x88\x0d'\
b'\x88\x0d\x9e\x0d\x9e\x0d\xb4\x0d\xb4\x0d\xca\x0d\xca\x0d\xf4\x0d'\

fontview = memoryview(_font)
indexview = memoryview(_index)

# Display driver

In [ ]:
%%micropython -pyboar 1

import pyb
from pyb import delay, SPI
from gc import collect
import time
from ufont import seven_segment, fill_rectangle, render
        
def colour565(rgb):
    # Given an iterable of 3, returns a 2-byte representing the 565 colour bytes
    c = ((rgb[0] & 0xF8) << 8) | ((rgb[1] & 0xFC) << 3) | (rgb[2] >> 3)
    return bytearray([c >> 8, (c & 0xff)])

def gray(self, frac):
    # Given a gray value between 0, and 1, returns a 2-byte representing the 565 colour bytes
    return colour565((int(255*frac), int(255*frac), int(255*frac)))

# a couple of colour constants 
# arguments are the standard RGB values
apple = colour565((96, 176, 65))
black = colour565((0, 0, 0))
blue = colour565((0, 0, 255))
green = colour565((0, 255, 0))
lightblue = colour565((20, 20, 25))
navy = colour565((0, 0, 128))
orange = colour565((255, 165, 0))
purple = colour565((128, 0, 128))
red = colour565((255, 0, 0))
yellow = colour565((255, 255, 0))
white = colour565((255, 255, 255))

class ILI9341:
    
    def __init__(self, spi, cs, dc, rst):
        self.baudrate = 21000000
        self.width = 240
        self.height = 320
        self._buffer_height = 5 # 5-6 seems to minimise the time for clear_screen
        self.buffer_length = self.width*self._buffer_height # length in 16-bit integers
        self.buffer = bytearray(2*self.buffer_length)
        self.mv = memoryview(self.buffer)
        self.dc = pyb.Pin(dc, mode=pyb.Pin.OUT) # data/command select pin
        self.cs = pyb.Pin(cs, mode=pyb.Pin.OUT) # chip select pin
        self.rst = pyb.Pin(rst, mode=pyb.Pin.OUT) # reset pin
        self.rst.high()
        self.spi = spi
        self.polarity = 0
        self.phase = 0
        self.reset()
        self.init()

    def spi_init(self):
        self.spi.init(SPI.MASTER, baudrate=self.baudrate, polarity=self.polarity, phase=self.phase)

    def reset(self):
        self.rst.low()
        time.sleep_ms(50)
        self.rst.high()
        time.sleep_ms(50)
        
    def write_data(self, data):
        self.spi_init()
        self.dc.high()
        self.cs.low()
        self.spi.send(data)
        self.cs.high()
        
    def write_command(self, command):
        self.spi_init()
        self.dc.low()
        self.cs.low()
        self.spi.send(command)
        self.cs.high()
        
    def init(self):
        
        self.write_command(0xEF)
        self.write_data(b'\x03\x80\x02')

        self.write_command(0xCF)
        self.write_data(b'\x00\xc1\x30')

        self.write_command(0xED)
        self.write_data(b'\x64\x03\x12\x81')

        self.write_command(0xE8)
        self.write_data(b'\x85\x00\x78')

        self.write_command(0xCB)
        self.write_data(b'\x39\x2c\x00\x34\x02')
        
        self.write_command(0xF7)
        self.write_data(b'\x20')

        self.write_command(0xEA)
        self.write_data(b'\x00\x00')

        self.write_command(0xC0)     #set power control 
        self.write_data(b'\x23')   #VRH[5:0] 

        self.write_command(0xC1);    #Power control 
        self.write_data(b'\x10')   #SAP[2:0];BT[3:0] 

        self.write_command(0xC5)    #VCM control 
        self.write_data(b'\x3e\x28')
        
        self.write_command(0xC7)    #VCM control2 
        self.write_data(b'\x86')

        # Note that this command determines the orientation of the screen, and setting a different
        # value makes possible to rotate or flip the display. See p. 201 of ILI9341.pdf
        self.write_command(0x36)    # set memory access control 
        self.write_data(b'\x88')

        self.write_command(0x3A)
        self.write_data(b'\x55')
        
        self.write_command(0xB1)
        self.write_data(b'\x00\x18')

        self.write_command(0xB6)    # set display function control 
        self.write_data(b'\x08\x82\x27')
        
        self.write_command(0xF2)     # disable gamma function
        self.write_data(b'\x00')

        self.write_command(0x26)    # select gamma curve
        self.write_data(b'\x01')
        
        self.write_command(0xE0)    # set gamma 
        self.write_data(b'\x0F\x31\x2b\x0c\x0e\x08\x4e\xf1\x37\x07\x10\x03\x0e\x09\x00')
        
        self.write_command(0xE1)    # set gamma 
        self.write_data(b'\x00\x0e\x14\x03\x11\x07\x31\xc1\x48\x08\x0f\x0c\x31\x36\x0f')

        self.write_command(0x11)     # exit sleep mode
        time.sleep_ms(120)
        self.write_command(0x29)   # display ON

    def address_window(self, x0, y0, x1, y1):
        # With this, we can save 300 us per call, because the SPI is not initialised 4 times
        # Assumes that the SPI was initialised properly
        self.dc.low()
        self.cs.low()
        self.spi.send(0x2A)
        self.dc.high()
        self.spi.send(bytes([(x0 >> 8), (x0 & 0xFF), (x1 >> 8), (x1 & 0xFF)]))
        self.dc.low()
        self.spi.send(0x2B) # row address set
        self.dc.high()
        self.spi.send(bytes([(y0>>8), (y0 & 0xFF), (y1>>8), (y1 & 0xFF)]))
        self.dc.low()
        self.spi.send(0x2C) # write data to RAM
        self.cs.high()
        
    def draw_pixel(self, x, y, colour):
        self.spi_init()
        self.address_window(x, y, x+1, y+1)
        self.write_data(colour)
        
    def draw_hline(self, x, y, w, colour):
        if (x < 0) or (x > self.width) or (y < 0) or (y > self.height): return
        w = min(w, self.width - x)
        self.spi_init()
        self.address_window(x, y, x+w-1, y)
        fill_rectangle(self.mv, colour, 0, 0, w, 1, w, 1)
        self.dc.high()
        self.cs.low()
        self.spi.send(self.mv[0:2*w])
        self.cs.high()

    def draw_vline(self, x, y, h, colour):
        if (x < 0) or (x > self.width) or (y < 0) or (y > self.height): return
        h = min(h, self.height - y)
        self.spi_init()
        self.address_window(x, y, x, y+h-1)
        fill_rectangle(self.mv, colour, 0, 0, 1, h, 1, h)
        self.dc.high()
        self.cs.low()
        self.spi.send(self.mv[0:2*h])
        self.cs.high()
        
    def draw_rect(self, x, y, w, h, rgb):
        # If speed is an issue, we could resort to an implimentation  
        # similar to that of draw_hline/draw_vline. That should save 
        # on the order of 1 ms per call. 
        if w <= 0 or h <= 0: return
        if (x < 0) or (x+w > self.width) or (y < 0) or (y+h > self.height): return
        self.draw_hline(x, y, w, rgb)
        self.draw_hline(x, y+h-1, w, rgb)
        self.draw_vline(x, y, h, rgb)
        self.draw_vline(x+w-1, y, h, rgb)
        
    def fill_rect(self, x, y, w, h, colour):
        fill_rectangle(self.mv, colour, 0, 0, self.buffer_length, 1, self.buffer_length, 1)
        H, M = divmod(w*h, self.buffer_length)
        self.spi_init()
        self.address_window(x, y, x+w-1, y+h-1)
        self.dc.high()
        self.cs.low()
        for i in range(H):
            self.spi.send(self.mv)
        if M > 0:
            self.spi.send(self.mv[0:2*M])
        self.cs.high()
        
    def clear(self, background=black):
        self.fill_rect(0, 0, self.width, self.height, background)

    def _show_seven_segment(self, digit, x0, y0, size, weight, fgcolour, bgcolour=black):
        height, width = seven_segment(self.mv, digit, size, weight, fgcolour, bgcolour)
        self.spi_init()
        self.address_window(x0, y0, x0+width-1, y0+height-1)
        self.dc.high()
        self.cs.low()
        self.spi.send(self.mv[0:2*width*height])
        self.cs.high()
        return width # return width, so that the character position can be incremented
        
    def digits(self, number, x0, y0, size, weight, fgcolour, bgcolour=black):
        # can display only seven-segment digits plus a decimal point, but it is fast
        self.spi_init()
        for digit in str(number):
            height, width = seven_segment(self.mv, digit, size, weight, fgcolour, bgcolour)
            #width = self._show_seven_segment(c, x0, y0, size, weight, fgcolour, bgcolour)
            self.address_window(x0, y0, x0+width-1, y0+height-1)
            self.dc.high()
            self.cs.low()
            self.spi.send(self.mv[0:2*width*height])
            self.cs.high()            
            x0 += (width+2*weight)

    def character(self, c, fv, iv):
        # retrieves the pattern corresponding to a particular character
        ordc = ord(c)
        ordc = ordc + 1 if ordc >= 32 and ordc <= 126 else 32
        idx_offs = 4 * (ordc - 32)
        offset = int.from_bytes(iv[idx_offs:idx_offs+2], 'little')
        next_offs = int.from_bytes(iv[idx_offs+2:idx_offs+4], 'little')
        width = int.from_bytes(fv[offset:offset+2], 'little')
        return fv[offset+2:next_offs], width
    
    def string(self, chars, fv, iv, x0, y0, height, foreground, background=black):
        # writes a string to the display, starting at position (x0, y0)
        # TODO: implement carriage return for long lines
        self.spi_init()
        x = x0
        for c in chars:
            f, _width = self.character(c, fv, iv)
            width = len(f)*8//height
            render(self.mv, bytearray(f), height, foreground, background)
            self.address_window(x, y0, x+width-1, y0+height-1)
            self.dc.high()
            self.cs.low()
            self.spi.send(self.mv[0:2*width*height])
            self.cs.high()
            x += _width

# pyboard tests

In [ ]:
%%micropython -pyboard 1

from utime import ticks_us, ticks_diff
from micropython import mem_info
from pyb import Pin, SPI
spi = SPI(1)

display = ILI9341(spi, 'C3', 'C1', 'C2')
collect()

mem_info()
print('==============================')
now = ticks_us()
display.clear()
print('screen clear: ', ticks_diff(ticks_us(), now), 'us')
print('')
mem_info()
print('==============================')
now = ticks_us()
display.digits(1234567890, 5, 55, 20, 2, red, black)
print('10 seven-segment digits with transfer: ', ticks_diff(ticks_us(), now), 'us')
print('')
mem_info()

print('==============================')
now = ticks_us()
for digit in '123456789.':
    seven_segment(display.mv, digit, 20, 3, red, black)
print('10 seven-segment digits without transfer: ', ticks_diff(ticks_us(), now), 'us')
print('')
mem_info()


print('==============================')
now = ticks_us()
display.string('micropython', fontview, indexview, 10, 150, 20, red, black)
print('11 characters with transfer: ', ticks_diff(ticks_us(), now), 'us')
print('')
mem_info()

print('==============================')
now = ticks_us()
for c in 'micropython':
    f, width = display.character(c, fontview, indexview)
    render(display.mv, bytearray(f), 20, red, black)
print('11 characters without transfer: ', ticks_diff(ticks_us(), now), 'us')
print('')
mem_info()

# one final garbage collection...
collect()
print('')
print('******************************')
mem_info()